In [1]:
import json
import pandas as pd

In [2]:
df_raw_log = pd.read_excel("data\es_smsLog.xlsx")
df_raw_list = pd.read_excel("data\es_smsSendList.xlsx")

<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:2: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:2: SyntaxWarning: invalid escape sequence '\e'
C:\Users\ybnoh\AppData\Local\Temp\ipykernel_25408\3713505422.py:1: SyntaxWarning: invalid escape sequence '\e'
  df_raw_log = pd.read_excel("data\es_smsLog.xlsx")
C:\Users\ybnoh\AppData\Local\Temp\ipykernel_25408\3713505422.py:2: SyntaxWarning: invalid escape sequence '\e'
  df_raw_list = pd.read_excel("data\es_smsSendList.xlsx")


# 1. 데이터 클랜징
1. df_log, df_list 필드 확인하여 불필요한 컬럼 제거
2. 각 컬럼의 값들 확인해서 적합한 데이터 타입으로 변경(df_log의 데이터 확인해서 각각 컬럼에 적합한 데이터 타입을 적용해줘)

In [3]:
log_columns = [
    "sno",
    "sendFl",
    "smsType",
    "sender",
    "contents",
    "receiverCnt",
    "receiverType",
    "sendStatus",
    "sendSuccessCnt",
    "sendFailCnt",
    "sendDt",
    "smsSendKey",
]
df_log = df_raw_log.copy()[log_columns]
# Convert columns to appropriate data types
df_log["sno"] = df_log["sno"].astype(int)
df_log["sendFl"] = df_log["sendFl"].astype("category")
df_log["smsType"] = df_log["smsType"].astype("category")
df_log["sender"] = df_log["sender"].astype(str)
df_log["contents"] = df_log["contents"].astype(str)
df_log["receiverCnt"] = df_log["receiverCnt"].astype(int)
df_log["receiverType"] = df_log["receiverType"].astype("category")
df_log["sendStatus"] = df_log["sendStatus"].astype("category")
df_log["sendSuccessCnt"] = df_log["sendSuccessCnt"].astype(int)
df_log["sendFailCnt"] = df_log["sendFailCnt"].astype(int)
df_log["sendDt"] = pd.to_datetime(df_log["sendDt"], errors="coerce")
df_log["smsSendKey"] = df_log["smsSendKey"].astype(int)

# 카카오 알림톡 필터링
df_log = df_log[df_log["sendFl"] != "kakao"]


# sender 데이터 다듬기
def extract_sender(string):
    arr = json.loads(string.replace("\\", ""))
    return arr[1]


df_log["sender"] = df_log["sender"].apply(extract_sender)

In [4]:
list_columns = [
    "sno",
    "smsLogSno",
    "receiverName",
    "receiverCellPhone",
    "sendCheckFl",
    "acceptCheckFl",
    "regDt",
    "kakaoSendKey",
]
df_list = df_raw_list.copy()[list_columns]
# Convert columns to appropriate data types
df_list["sno"] = df_list["sno"].astype(int)
df_list["smsLogSno"] = df_list["smsLogSno"].astype(int)
df_list["receiverName"] = df_list["receiverName"].astype(str)
df_list["receiverCellPhone"] = df_list["receiverCellPhone"].astype(str)
df_list["sendCheckFl"] = df_list["sendCheckFl"].astype("category")
df_list["acceptCheckFl"] = df_list["acceptCheckFl"].astype("category")
df_list["regDt"] = pd.to_datetime(df_list["regDt"], errors="coerce")
df_list["kakaoSendKey"] = df_list["kakaoSendKey"].astype(float)

# 카카오 알림톡 필터링
df_list = df_list[df_list.kakaoSendKey.isna()]
df_list = df_list.drop("kakaoSendKey", axis=1)
df_list

,sno,smsLogSno,receiverName,receiverCellPhone,sendCheckFl,acceptCheckFl,regDt
0,430691,75770,이남헌,1083800903,y,y,2024-08-01 00:28:45
1,430692,75770,구매물류관리자,1099361333,y,y,2024-08-01 00:28:45
2,430693,75770,구매물류관리자,1028485625,y,y,2024-08-01 00:28:45
3,430694,75770,박수진,1037191851,y,y,2024-08-01 00:28:45
4,430695,75770,이동석,1097888457,y,y,2024-08-01 00:28:45
...,...,...,...,...,...,...,...
78906,509637,90310,김승재,1065472231,r,y,2024-11-11 16:46:13
78907,509638,90310,고동민,1076161680,r,y,2024-11-11 16:46:13
78908,509639,90310,박진혁,1043889221,r,y,2024-11-11 16:46:13
78909,509640,90310,손민지,1028463252,r,y,2024-11-11 16:46:13


In [5]:
# df_log와 df_list에서 전송실패한 로그 갯수 같은지 확인
# 전송 안되는 케이스는 대부분 수신번호가 이상한 경우
print(df_log.sendFailCnt.sum())
df_list[df_list.sendCheckFl == "n"]

169


,sno,smsLogSno,receiverName,receiverCellPhone,sendCheckFl,acceptCheckFl,regDt
751,431442,75904,플닥,1000000000,n,y,2024-08-01 15:43:37
3443,434134,76403,df,1000000000,n,y,2024-08-05 14:31:56
3461,434152,76406,vv,1000000000,n,y,2024-08-05 14:34:25
3470,434161,76409,vv,1000000000,n,y,2024-08-05 14:36:28
3572,434263,76427,plan,1000000000,n,y,2024-08-05 15:21:52
...,...,...,...,...,...,...,...
77134,507865,90004,ITMVIP대표회원,1000000000,n,y,2024-11-08 17:25:08
77370,508101,90042,한규남,25383450,n,y,2024-11-08 19:01:38
77887,508618,90128,한규남,25383450,n,y,2024-11-09 23:35:18
77899,508630,90131,김태연,1091022644,n,y,2024-11-10 02:57:39


# 11월 1일 데이터 전수조사
결론: 결과수신대기 상태는 메시지 전송 성공으로 봐야 함

In [6]:
df_log_1104 = df_log[df_log["sendDt"].dt.date == pd.to_datetime("2024-11-04").date()]
df_log_1104 = df_log_1104.sort_values(by="sendDt", ascending=False)
df_log_1104

,sno,sendFl,smsType,sender,contents,receiverCnt,receiverType,sendStatus,sendSuccessCnt,sendFailCnt,sendDt,smsSendKey
13391,89193,lms,member,SYSTEM,[플랜닥스]\n회원님의 인증번호는 [21502106]입니다.\n\n감사합니다.\n스...,1,each,r,0,0,2024-11-04 22:56:30,17307285909465
13390,89192,sms,member,SYSTEM,[플랜닥스]\n김교륭회원님이 회원가입하셨습니다!,7,group,r,0,0,2024-11-04 20:55:20,17307213202867
13389,89191,lms,member,SYSTEM,[플랜닥스]\n김교륭 회원님의 가입을 진심으로 축하드립니다.\n\n회원승인은 첨부서...,1,each,r,0,0,2024-11-04 20:55:20,17307213204829
13388,89190,sms,order,SYSTEM,관리자\n조현교님의 2411041949463470 주문이 결제완료 되었습니다.,10,group,r,0,0,2024-11-04 19:50:50,17307174501301
13387,89189,sms,order,SYSTEM,관리자\n정규식님의 2411041950147743 주문이 결제완료 되었습니다.,10,group,r,0,0,2024-11-04 19:50:15,17307174153165
...,...,...,...,...,...,...,...,...,...,...,...,...
13111,88911,lms,user,plandocs_cx,"[플랜닥스]\n{restockName} 고객님,\n알림을 요청하신\n{restock...",1,NaN,r,0,0,2024-11-04 08:32:38,17306767582551
13101,88901,lms,member,SYSTEM,[플랜닥스]\n이민혁님의 플닥페이 -201원이 차감되었습니다.\n\n현재 플닥페이 ...,1,each,r,0,0,2024-11-04 00:24:22,17306474628359
13102,88902,sms,order,SYSTEM,관리자\n이민혁님의 2411040024050779 주문이 결제완료 되었습니다.,10,group,r,0,0,2024-11-04 00:24:22,17306474622933
13100,88900,sms,order,SYSTEM,관리자\n이민혁님의 2411040023209934 주문이 결제완료 되었습니다.,10,group,r,0,0,2024-11-04 00:23:41,17306474213903


In [7]:
df_log_1104[df_log_1104.sendStatus == "n"]

,sno,sendFl,smsType,sender,contents,receiverCnt,receiverType,sendStatus,sendSuccessCnt,sendFailCnt,sendDt,smsSendKey
13272,89074,sms,order,SYSTEM,관리자\n신기철님의 2411041459372880 주문이 결제완료 되었습니다.,10,group,n,8,0,2024-11-04 14:59:38,17306999785697
13270,89072,sms,order,SYSTEM,관리자\n최동헌님의 2411041452184141 주문이 결제완료 되었습니다.,10,group,n,9,0,2024-11-04 14:52:20,17306995408364
13268,89070,sms,order,SYSTEM,관리자\n최호성님의 2411041447300770 주문이 결제완료 되었습니다.,10,group,n,9,0,2024-11-04 14:51:43,17306995033644
13170,88972,sms,order,plandocs_it3,[플랜닥스]\n마이페이지테스트플러스님의 2411041039552404 주문이 환불 ...,1,each,n,0,1,2024-11-04 10:40:59,17306844595613


In [8]:
df_list[df_list.smsLogSno == 89192]

,sno,smsLogSno,receiverName,receiverCellPhone,sendCheckFl,acceptCheckFl,regDt
72711,503434,89192,이남헌,1083800903,r,y,2024-11-04 20:55:20
72712,503435,89192,박수진,1037191851,r,y,2024-11-04 20:55:20
72713,503436,89192,이동석,1097888457,r,y,2024-11-04 20:55:20
72714,503437,89192,정정필,1050191019,r,y,2024-11-04 20:55:20
72715,503438,89192,고동민,1076161680,r,y,2024-11-04 20:55:20
72716,503439,89192,손민지,1028463252,r,y,2024-11-04 20:55:20
72717,503440,89192,이시현,1020059683,r,y,2024-11-04 20:55:20
